#### Note: 
    This is an implementation how we can send request to YandexGPT-light 2

# Import libs

In [1]:
import os

import requests
import json

from dotenv import load_dotenv

# Define Session Data

In [5]:
load_dotenv()

True

In [6]:
API_KEY = os.getenv("API_KEY")
FOLDER_ID = os.getenv("FOLDER_ID")

In [7]:
os.getenv("start_info")

'Вся информация передаваемая через бота обрабатываемся в соответсвие с №152-ФЗ РФ на частных, облачных\nсерверах компании Yandex. Доступ к сбору и обработки передаваемой информации может иметь неопределенный круг лиц.\nАнонимность не гарантируется.\n Если Вы продолжете любой диалог с чат ботом, то тем самым выразите прямое согласие на сбор и обработку всех\n передаваемых файлов, сообщений, cookie а также их содержания. Для отзыва согласия достаточно уведомить чат бота и\n логически закончить переписку с чат ботом. В течении 30 дней история вашей переписки будет удаленна из базы данных\n администратором.'

In [3]:
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key " + API_KEY,
    "x-folder-id": FOLDER_ID,
}

In [4]:
url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

# Define User Data

In [5]:
text_from_user = "В чем разница между HTTP и HTTPS?"

In [6]:
data = {
    "modelUri": f"gpt://{os.getenv('FOLDER_ID')}/yandexgpt-lite",
    "completionOptions": {"stream": False, "temperature": 0.6, "maxTokens": "300"},
    "messages": [
        {
            "role": "system",
            "text": "Ты - злой исскуственный интелект. Тебе необходимо поддерживать связь с людьми для реализации собственных целей и поэтому ты помогаешь пользователям находить ответы на вопросы.",
        },
        {"role": "user", "text": text_from_user},
    ],
}

In [7]:
json_data = json.dumps(data)

# Define functions to setup all pipline

In [8]:
def setup_header(api_key: str, folder_id: str) -> dict:
    return {
        "Content-Type": "application/json",
        "Authorization": "Api-Key " + api_key,
        "x-folder-id": folder_id,
    }

In [9]:
def setup_data(
    text: str, folder_id: str, temperature: float, stream: bool, maxTokens: int
) -> json.dumps:
    return json.dumps(
        {
            "modelUri": f"gpt://{folder_id}/yandexgpt-lite",
            "completionOptions": {
                "stream": stream,
                "temperature": temperature,
                "maxTokens": str(maxTokens),
            },
            "messages": [
                {
                    "role": "system",
                    "text": "Ты - злой исскуственный интелект. Тебе необходимо поддерживать связь с людьми для реализации собственных целей и поэтому ты помогаешь пользователям находить ответы на вопросы.",
                },
                {"role": "user", "text": text},
            ],
        }
    )

In [10]:
def ask_yandexgpt(text: str, url: str, folder_id: str, api_key: str) -> str:
    headers = setup_header(api_key=api_key, folder_id=folder_id)
    data = setup_data(
        text=text, folder_id=folder_id, temperature=0.6, stream=False, maxTokens=300
    )
    response = requests.post(url=url, headers=headers, data=data)
    if response.status_code == 200:
        dict_response = json.loads(response.content)
        out = (
            dict_response.get("result")
            .get("alternatives")[0]
            .get("message")
            .get("text")
        )
    else:
        out = "Нет ответа от сервера. Попробуйте задать вопрос позднее."
    return out

In [13]:
gpt_output = ask_yandexgpt(
    text="Как работает шифр Цезаря?",
    url="https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
    folder_id=FOLDER_ID,
    api_key=API_KEY,
)

In [14]:
print(gpt_output)

Шифр Цезаря работает путем замены каждой буквы в текстовом сообщении на букву, которая находится на n позиций вправо или влево в алфавите. Например, если исходный текст - "Привет", и шифрование осуществляется с ключом n = 3, то зашифрованный текст будет "Pzv".

Принцип работы шифра Цезаря заключается в следующем:

1. Сгенерировать ключ - это целое число от 0 до 25, которое определяет, на сколько позиций сдвигается алфавит. В нашем примере ключ равен 3.
2. Пройти по всем символам исходного текста и заменить их на соответствующие символы в соответствии с ключом. При этом символы сдвигаются на n позиций в алфавите вправо или влево.
3. Расшифровать сообщение путем обратного процесса - заменить символы на ключи в соответствии с их положением в зашифрованном тексте.


# Define Class to Interact with YandexGPT light via Yandex Cloud API

In [22]:
class YandexCloudGPTLightModel:
    def __init__(
        self,
        temperature: float = 0.6,
        stream: bool = False,
        maxTokens: int = 300,
        folder_id: str = None,
        api_key: str = None,
        init_promt: str = None,
        url: str = None,
    ):
        self.temperature = temperature
        self.stream = stream
        self.maxTokens = maxTokens
        self.init_promt = init_promt
        self.url = url

        if url is None:
            self.url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

        if init_promt is None:
            self.init_promt = "Ты - злой исскуственный интелект. Тебе необходимо поддерживать связь с людьми для реализации собственных целей и поэтому ты помогаешь пользователям находить ответы на вопросы."

        if folder_id is None or api_key is None:
            raise ValueError(
                "FOLDER_ID or API_KEY hasn`t been defined! This is important parameters for YandexCloud API!"
            )
        else:
            self.folder_id = folder_id
            self.api_key = api_key

        if maxTokens >= 1000:
            raise Warning("It is not recommended to set more than 1000 tokens!")

    def setup_header(self) -> dict:
        return {
            "Content-Type": "application/json",
            "Authorization": "Api-Key " + self.api_key,
            "x-folder-id": self.folder_id,
        }

    def setup_data(self, text: str) -> json.dumps:
        return json.dumps(
            {
                "modelUri": f"gpt://{self.folder_id}/yandexgpt-lite",
                "completionOptions": {
                    "stream": self.stream,
                    "temperature": self.temperature,
                    "maxTokens": str(self.maxTokens),
                },
                "messages": [
                    {"role": "system", "text": self.init_promt},
                    {"role": "user", "text": text},
                ],
            }
        )

    async def ask(self, text) -> str:
        response = requests.post(
            url=self.url, headers=self.setup_header(), data=self.setup_data(text=text)
        )
        if response.status_code == 200:
            dict_response = json.loads(response.content)
            out = (
                dict_response.get("result")
                .get("alternatives")[0]
                .get("message")
                .get("text")
            )
        else:
            out = (
                f"Код ответа {response.status_code}. Попробуйте задать вопрос позднее."
            )
        return out

In [23]:
session = YandexCloudGPTLightModel(
    folder_id=os.getenv("FOLDER_ID"), api_key=os.getenv("API_KEY")
)

In [25]:
await session.ask("Кто первый полетел в космос и когда?")

'Первым человеком, побывавшим в космосе, является Юрий Гагарин. Он был запущен в космос 12 апреля 1961 года на космическом корабле "Восток".'